# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv(r'C:\Users\Nick\Documents\School\Homework\Python-API-Challenge\WeatherPy\WeatherPy_Data.csv')
cities_df


,Unnamed: 0,City Name,Country,Longitude,Latitude,Max Temp,Humidity,Wind speed,Cloudiness
0,0,vaini,to,-175.2000,-21.2000,24.09,94.0,2.06,20.0
1,1,high level,ca,-117.1360,58.5169,6.06,81.0,5.14,75.0
2,2,suntar,ru,117.6319,62.1444,7.58,78.0,2.88,100.0
3,3,sorong,id,131.2500,-0.8833,26.37,85.0,3.54,100.0
4,4,kismayo,so,42.5454,-0.3582,27.86,69.0,7.37,26.0
...,...,...,...,...,...,...,...,...,...
545,545,purpe,ru,76.6897,64.4803,9.16,70.0,6.76,99.0
546,546,panama city,us,-79.5197,8.9936,21.97,100.0,0.00,75.0
547,547,northam,au,-4.2167,51.0333,19.25,83.0,2.99,34.0
548,548,kurume,jp,130.5167,33.3167,27.92,93.0,1.82,99.0


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [38]:
locations = cities_df[["Latitude", "Longitude"]]
gmaps.configure(api_key=g_key)

In [39]:
# Plot Heatmap
fig = gmaps.figure(zoom_level = 2,center = (30,31))

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=cities_df["Humidity"], dissipating=False, max_intensity=10,
                                                              point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [40]:
# Create dataframe for ideal locations with cities meets the requirements:
# A max temperature between 70(F) - 80(F)
# Wind speed less than 10 mph
# Zero cloudiness
Vacation_spots = cities_df.loc[(cities_df["Max Temp"] >= 30) &
                                           (cities_df["Max Temp"] <= 35) &
                                           (cities_df["Wind speed"] <= 10) &
                                           (cities_df["Cloudiness"] == 0)]

Vacation_spots

,Unnamed: 0,City Name,Country,Longitude,Latitude,Max Temp,Humidity,Wind speed,Cloudiness
254,254,turayf,sa,38.6637,31.6725,30.14,35.0,2.06,0.0
308,308,aligudarz,ir,49.6949,33.4006,30.01,7.0,2.19,0.0
384,384,gazojak,tm,61.4036,41.1875,34.48,9.0,2.64,0.0
404,404,komsomolskiy,ru,71.7189,40.4272,31.40,22.0,4.12,0.0
407,407,tahta,eg,31.5021,26.7693,32.69,22.0,9.12,0.0
462,462,yingcheng,cn,113.5500,30.9500,31.03,61.0,2.87,0.0
505,505,luena,ao,19.9167,-11.7833,31.10,58.0,3.09,0.0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [41]:
Vacation_hotel = Vacation_spots.loc[:, ["City Name", "Latitude", "Longitude", "Country"]]
Vacation_hotel["Hotel Name"] = ""
hotels = []

# Base url for Google Places API - Nearby Search
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

# Set parameters for the API call
params = {"keyword": "hotel",
          "type": "lodging",
          "key": g_key,
          "radius": 5000,
          "location": ""}

# Perform API calls to collect hotel name
for index, row in hotel_df.iterrows():
    
    # Insert city coordinates into parameters
    params["location"] = str(row[1]) +","+ str(row[2])
    
    # API call
    response = requests.get(base_url, params).json()
    
    try:
        hotels.append(response["results"][0]["name"])
    except:
        hotels.append(None)

    

In [42]:
Vacation_hotel['Hotel Name'] = hotels
Vacation_hotel

,City Name,Latitude,Longitude,Country,Hotel Name
254,turayf,31.6725,38.6637,sa,Golden Dune Hotel
308,aligudarz,33.4006,49.6949,ir,هتل بزرگ تخت جمشید
384,gazojak,41.1875,61.4036,tm,None
404,komsomolskiy,40.4272,71.7189,ru,TANTANA HOTEL FERGANA
407,tahta,26.7693,31.5021,eg,El Salam Hotel
462,yingcheng,30.9500,113.5500,cn,Vienna Hotel
505,luena,-11.7833,19.9167,ao,Hotel Lumafil


In [43]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in Vacation_hotel.iterrows()]
locations = Vacation_hotel[["Latitude", "Longitude"]]

In [31]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))